In [1]:
import pandas as pd
import glob
import re
import csv
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.integrate import simps
from numpy import trapz
from sklearn.metrics import auc
import math


paths = ["/home/seb300/QLearningAgentTransitionFunction/learnability-Ttest",
         "/home/seb300/QLearningAgentTransitionFunction/generalization-Ttest"]

#paths = ["/content/drive/MyDrive/master thesis/Pacman/generalization"]
file_dict={}
file_dict_results = {}

for path in paths:
  %cd $path
  method = path.split("/")[-1]
  if method not in file_dict:
    file_dict[method]={}
  for filename in glob.glob("./*"):
    # print(filename)
    #./learnability2agents_BoltzmannAgent_v3M_RandomGhost_{"index":1,"prob":{}}_{"mean":0,"std":0}_v3M_RandomGhost_{"index":1,"prob":{}}_{"mean":0,"std":0}.pkl
    elements = filename.replace(".pkl","").split("_")
    typeofagent = elements[1]
    envs = re.split(r'{"mean".*?}', "".join(elements[3:]))
    noises = re.findall(r'{"mean".*?}', filename)
    if "ensemble" in method:
      testenv = envs[0]
      noise_testenv = noises[0]
      try:
        trainenv = envs[1]
      except:
        trainenv = ""
      noise_trainenv = noises[1]
    elif "generalization" in method:
      try:
        testenv = envs[1]
      except:
        testenv = ""
      noise_testenv = noises[1]
      trainenv = envs[0]
      noise_trainenv = noises[0]
    else:
      testenv = envs[0]
      noise_testenv = noises[0]
      trainenv = envs[0]
      noise_trainenv = noises[0]
    if typeofagent not in file_dict[method]:
      file_dict[method][typeofagent] = {}
    if testenv not in file_dict[method][typeofagent]:
      file_dict[method][typeofagent][testenv] = {}
    if noise_testenv not in file_dict[method][typeofagent][testenv]:
      file_dict[method][typeofagent][testenv][noise_testenv] = {}
    if trainenv not in file_dict[method][typeofagent][testenv][noise_testenv]:
      file_dict[method][typeofagent][testenv][noise_testenv][trainenv] = {}
    if noise_trainenv not in file_dict[method][typeofagent][testenv][noise_testenv][trainenv]:
      file_dict[method][typeofagent][testenv][noise_testenv][trainenv][noise_trainenv] = {}
    with open(filename, newline='') as csvfile:
      if "errorbar" in filename:
        file_dict[method][typeofagent][testenv][noise_testenv][trainenv][noise_trainenv]["errorbar"] = []
      else:
        file_dict[method][typeofagent][testenv][noise_testenv][trainenv][noise_trainenv]["values"] = []
      reader = csv.reader(csvfile, delimiter='\n', quotechar='|')
      for row in reader:
        list_el = []
        for el in row[0].split(","):
          list_el.append(float(el))
        if "errorbar" in filename:
          file_dict[method][typeofagent][testenv][noise_testenv][trainenv][noise_trainenv]["errorbar"].append(float(row[0]))
        else:
          file_dict[method][typeofagent][testenv][noise_testenv][trainenv][noise_trainenv]["values"].append(float(row[0]))

/home/seb300/miniconda3/envs/.py39env/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/seb300/QLearningAgentTransitionFunction/learnability-Ttest
/home/seb300/QLearningAgentTransitionFunction/generalization-Ttest


In [2]:
import scipy
def TTest(x,y):
    return x.mean(), y.mean(), scipy.stats.ttest_ind(x,y)

In [3]:
file_dict['generalization-Ttest']['BoltzmannAgent']['v2DirectionalGhost{"index":1,"prob":0.3}']['{"mean":0,"std":0.1}']['v2DirectionalGhost{"index":1,"prob":0.3}'].keys()

dict_keys(['{"mean":0,"std":0}'])

In [4]:
# NON SEMANTIC NOISE
#grid
somegrid_l = ['v2','v3','v4']
someagent_l = ['SarsaAgent','BoltzmannAgent']
someexploration_strategy_l = ['Boltzmann','Sarsa']
#training env
someghost_l = ['RandomGhost{"index":1,"prob":{}}','DirectionalGhost{"index":1,"prob":0.3}','DirectionalGhost{"index":1,"prob":0.6}']
somenoise_l = ['{"mean":0,"std":0}','{"mean":0,"std":0}','{"mean":0,"std":0}']
#testing env
someotherghost_l = ['RandomGhost{"index":1,"prob":{}}','DirectionalGhost{"index":1,"prob":0.3}','DirectionalGhost{"index":1,"prob":0.6}']
someothernoise_l = ['{"mean":0,"std":0.1}','{"mean":0,"std":0.1}','{"mean":0,"std":0.1}']

for someagent in someagent_l:
    for someexploration_strategy in someexploration_strategy_l:
        for somegrid in somegrid_l:
            for someghost, somenoise, someotherghost, someothernoise in zip(someghost_l, somenoise_l, someotherghost_l, someothernoise_l):
                print(someagent, someexploration_strategy, f"{somegrid}{someghost}", somenoise, f"{somegrid}{someotherghost}", someothernoise)
                G =  file_dict['generalization-Ttest'][someagent][f"{somegrid}{someghost}"][someothernoise][f"{somegrid}{someghost}"][somenoise]["values"]
                L =  file_dict['learnability-Ttest'][someagent][f"{somegrid}{someghost}"][someothernoise][f"{somegrid}{someghost}"][someothernoise]["values"]
                print(TTest(np.asarray(G),np.asarray(L)))          

SarsaAgent Boltzmann v2RandomGhost{"index":1,"prob":{}} {"mean":0,"std":0} v2RandomGhost{"index":1,"prob":{}} {"mean":0,"std":0.1}
(256.6254001030922, 187.99599955077775, TtestResult(statistic=7.475710871990846, pvalue=1.676137613429319e-13, df=999.0))
SarsaAgent Boltzmann v2DirectionalGhost{"index":1,"prob":0.3} {"mean":0,"std":0} v2DirectionalGhost{"index":1,"prob":0.3} {"mean":0,"std":0.1}
(255.39380001538993, 201.03520000975826, TtestResult(statistic=6.010238296200062, pvalue=2.5934155709930306e-09, df=999.0))
SarsaAgent Boltzmann v2DirectionalGhost{"index":1,"prob":0.6} {"mean":0,"std":0} v2DirectionalGhost{"index":1,"prob":0.6} {"mean":0,"std":0.1}
(285.15660020935536, 248.41680098522804, TtestResult(statistic=4.430342815710423, pvalue=1.0446995386878753e-05, df=999.0))
SarsaAgent Boltzmann v3RandomGhost{"index":1,"prob":{}} {"mean":0,"std":0} v3RandomGhost{"index":1,"prob":{}} {"mean":0,"std":0.1}
(96.56740023326874, 46.2883999438105, TtestResult(statistic=5.0074479919293715, pv

In [5]:
# SEMANTIC NOISE
#grid
somegrid_l = ['v2','v3','v4']
someagent_l = ['SarsaAgent','BoltzmannAgent']
someexploration_strategy_l = ['Boltzmann','Sarsa']
#training env
someghost_l = ['RandomGhost{"index":1,"prob":{}}']
somenoise_l = ['{"mean":0,"std":0}']
#testing env
someotherghost_l = ['RandomGhostTeleportingNearWalls{"index":1,"prob":{}}']
someothernoise_l =  ['{"mean":0,"std":0}']

for someagent in someagent_l:
    for someexploration_strategy in someexploration_strategy_l:
        for somegrid in somegrid_l:
            for someghost, somenoise, someotherghost, someothernoise in zip(someghost_l, somenoise_l, someotherghost_l, someothernoise_l):
                print(someagent, someexploration_strategy, f"{somegrid}{someghost}", somenoise, f"{somegrid}{someotherghost}", someothernoise)
                G =  file_dict['generalization-Ttest'][someagent][f"{somegrid}{someghost}"][someothernoise][f"{somegrid}{someghost}"][somenoise]["values"]
                L =  file_dict['learnability-Ttest'][someagent][f"{somegrid}{someghost}"][someothernoise][f"{somegrid}{someghost}"][someothernoise]["values"]
                print(TTest(np.asarray(G),np.asarray(L)))          

SarsaAgent Boltzmann v2RandomGhost{"index":1,"prob":{}} {"mean":0,"std":0} v2RandomGhostTeleportingNearWalls{"index":1,"prob":{}} {"mean":0,"std":0}
(491.69719995117185, 485.50000100469396, TtestResult(statistic=6.112140017936378, pvalue=1.415060227677934e-09, df=984.0))
SarsaAgent Boltzmann v3RandomGhost{"index":1,"prob":{}} {"mean":0,"std":0} v3RandomGhostTeleportingNearWalls{"index":1,"prob":{}} {"mean":0,"std":0}
(423.2652000427246, 438.56795623947903, TtestResult(statistic=-5.368935861555879, pvalue=1.019562566859676e-07, df=860.0))
SarsaAgent Boltzmann v4RandomGhost{"index":1,"prob":{}} {"mean":0,"std":0} v4RandomGhostTeleportingNearWalls{"index":1,"prob":{}} {"mean":0,"std":0}
(370.61270867834486, 411.8459714916645, TtestResult(statistic=-6.6958727238380416, pvalue=4.45056699181866e-11, df=689.0))
SarsaAgent Sarsa v2RandomGhost{"index":1,"prob":{}} {"mean":0,"std":0} v2RandomGhostTeleportingNearWalls{"index":1,"prob":{}} {"mean":0,"std":0}
(491.69719995117185, 485.50000100469396